# USG grants crawl
## Ingest

### Motive

Imagine that we were curious about how the federal governemnt had fostered or otherwise encouraged [open science](https://open.science.gov/) and associated infrastructure in recent years.  How would we even begin to explore this issue?

One potential resource could might be [grants.gov](https://www.grants.gov/web/grants) which serves as an online data resource for government grants in the United States (primarily federal, but also some state).  With this resource it's possible to [explore details about federal grants](https://www.grants.gov/web/grants/search-grants.html), including which agencies are offering them, what they are targeting, and how much funding is available.  It's also possible to [download](https://www.grants.gov/xml-extract.html) much (but not all) of this database for local use.

### Initial database load

Let's begin by loading up the database provided by the website, which is stored in an xml format.

**Notebook collection-wide note**:  Throughout this collection of notebooks, whenever we first introduce an approach or functionality we will tend to present the majority (if not the entirety) of the code used to produce the results that are being displayed and explored.  This is done for the sake of transparency and learning.  Subsequent applications of established code / functionality will tend to make use of a script collection, `grantsGov_utilities.py`, which is contained in the `src/` directory of this repository.

In [1]:
from bs4 import BeautifulSoup
import xmltodict
import sys

# FUTURE NOTE: it may be possible to do a check for a local file meeting the relevant criterion and conditionally 
# download from https://www.grants.gov/extract/ (and extract compressed file) in the event a local target isn't found.
# For the moment though...

# load up the xml file; hard-path to local file.  Adjust as necessary
pathToXML='C://Users//dbullock//Documents//code//gitDir//USG_grants_crawl//inputData//GrantsDBExtract20230113v2.xml'

# open and parse file
with open(pathToXML, 'r') as f:
    govGrantData_raw = f.read()

# convert xml to dictionary
with open(pathToXML) as xml_file:
    govGrantData_dictionary = xmltodict.parse(xml_file.read())

# quick size legibility function generated by code-davinci-002
def convert_bytes(bytes):
    if bytes < 1024:
        return str(bytes) + " B"
    elif bytes < 1048576:
        return str(round(bytes/1024, 1)) + " KB"
    elif bytes < 1073741824:
        return str(round(bytes/1048576, 1)) + " MB"
    elif bytes < 1099511627776:
        return str(round(bytes/1073741824, 1)) + " GB"
    else:
        return str(round(bytes/1099511627776, 1)) + " TB"
    
# terminal reports
print('Dictionary conversion successful')
print('\n' + str(len(govGrantData_dictionary['Grants']['OpportunitySynopsisDetail_1_0'])) + ' grant entries found, totalling '+ convert_bytes(sys.getsizeof(govGrantData_raw)))
#print('\n and with dictionary keys:\n')
#print(govGrantData_dictionary['Grants']['OpportunitySynopsisDetail_1_0'][0].keys())

Dictionary conversion successful

70330 grant entries found, totalling 256.2 MB


### What does a grant record look like

To get a sense of what any give grant records looks like in the XML structure / metadata scheme / python dictionary, we can select one arbitrarly and view it.

In [2]:
print("{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in  govGrantData_dictionary['Grants']['OpportunitySynopsisDetail_1_0'][1].items()) + "}")

{'OpportunityID': '262149',
'OpportunityTitle': 'Eradication of Yellow Crazy Ants on Johnston Atoll NWR',
'OpportunityNumber': 'F14AS00402',
'OpportunityCategory': 'D',
'FundingInstrumentType': 'CA',
'CategoryOfFundingActivity': ['AG', 'ENV', 'NR'],
'CFDANumbers': '15.608',
'EligibleApplicants': '99',
'AdditionalInformationOnEligibility': 'The recipient has already been selected for this award.  Please see attached Notice of Intent to Award for specifics.',
'AgencyCode': 'DOI-FWS',
'AgencyName': 'Fish and Wildlife Service',
'PostDate': '08152014',
'CloseDate': '08222014',
'LastUpdatedDate': '08152014',
'AwardCeiling': '0',
'AwardFloor': '0',
'EstimatedTotalProgramFunding': '0',
'Description': 'Funds under this award are to be used for the eradication of Yellow Crazy Ants from Johnston Atoll National Wildlife Refuge.',
'Version': 'Synopsis 1',
'CostSharingOrMatchingRequirement': 'No',
'ArchiveDate': '08232014',
'AdditionalInformationURL': 'http://www.grants.gov/',
'AdditionalInformation

### A quick conversion and freeing up memory

We're not going to need all of the information that's contained in the XML database, at least not for the majority of the investigations we'll be pursuing.  Additionally, indexing into an unstructured data object like this likely isn't the most resource or time efficient approach.  As such, let's go ahead and get the data fields we _do_ want, and drop the rest (to free up memory).


In [3]:
import pandas as pd
# convert to pandas dataframe
grantsDF=pd.DataFrame.from_records(govGrantData_dictionary['Grants']['OpportunitySynopsisDetail_1_0'], columns=['OpportunityID', 'OpportunityTitle','OpportunityNumber','AgencyCode', 'AgencyName', 'AwardCeiling', 'AwardFloor', 'EstimatedTotalProgramFunding', 'ExpectedNumberOfAwards', 'Description'])
# proactively replace nans for agency name with other
# replace the nans
grantsDF=grantsDF.fillna(0)
# free up memory
del govGrantData_dictionary

grantsDF.head(50)

,OpportunityID,OpportunityTitle,OpportunityNumber,AgencyCode,AgencyName,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,DOS-SA,Bureau of South and Central Asian Affairs,600000,400000,600000,1,The Office of Press and Public Diplomacy of th...
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,DOI-FWS,Fish and Wildlife Service,0,0,0,0,Funds under this award are to be used for the ...
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,DOI-USGS1,Geological Survey,0,0,31900,1,The USGS Southeast Ecological Science Center s...
3,131094,Plant Feedstock Genomics for Bioenergy: A Joi...,DE-FOA-0000598,PAMS-SC,Office of Science,500000,200000,6000000,10,The U.S. Department of Energy&apos;s Office of...
4,131095,Management of HIV-Related Lung Disease and Car...,RFA-HL-12-034,HHS-NIH11,National Institutes of Health,400000,0,2000000,0,This FOA invites clinical trials planning gran...
5,131093,Shared Instrumentation Grant Program (S10),PAR-12-017,HHS-NIH11,National Institutes of Health,0,0,43000000,110,The NCRR Shared Instrument Grant (SIG) program...
6,262168,Idaho BLM - Croy Creek Trail,L14AS00315,DOI-BLM,Bureau of Land Management,95000,0,95000,1,Reestablish the Croy Creek Trail Network to al...
7,262188,"Assessment of Population, Reproductive, and He...",F14AS00404,DOI-FWS,Fish and Wildlife Service,250000,150000,250000,1,This program will provide assistance to an ins...
8,262189,Office of Elementary and Secondary Education (...,ED-GRANTS-081814-001,ED,Department of Education,0,0,80000000,5,Note: Each funding opportunity description is ...
9,262190,Office of Elementary and Secondary Education (...,ED-GRANTS-081814-002,ED,Department of Education,0,0,160000000,7,Note: Each funding opportunity description is ...


### Does the data need cleaning?

As with every case when working with a data source, we need to ask ourselves if our data is reliable and/or if it needs cleaning.  As it turns out, we do indeed need to clean this data.  For example, all of the numbers in our dataframe are currently considered strings, which will not be useful if we want to do math later.

In a number of instances, there no information entered in the fields relating to the agency identity.  We'll have to group these in an `Other` category, for now.  Additionally the AgencyCode field likely has more information than we need, so we can fix that too.

One of the major issues though, is the incorrect information being entered in the `ExpectedNumberOfAwards` field.  For grants which do not have a `EstimatedTotalProgramFunding`, we use this in combination with the grant ceiling and floor to estimate what this ought to be.  When an unreliable number is entered here it can really throw off later computations.  We won't change this at the moment, but instead implement a threshold beyond which we won't trust the number.

NOTE: A much more comprehensive and robust cleaning approach is implemented in `grantsGov_utilities.prepareGrantsDF` and `grantsGov_utilities.repairFunding_GovGrantsDf`

In [4]:
# let's do some initial dataset overview
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from itertools import compress
from IPython.display import display, HTML

grantsDF[['AwardCeiling','AwardFloor','EstimatedTotalProgramFunding','ExpectedNumberOfAwards']]=grantsDF[['AwardCeiling','AwardFloor','EstimatedTotalProgramFunding','ExpectedNumberOfAwards']].astype(np.int64)

display((grantsDF[['OpportunityID','AgencyName','EstimatedTotalProgramFunding','AwardCeiling','AwardFloor','ExpectedNumberOfAwards']].sort_values(by='ExpectedNumberOfAwards')).iloc[-50:])


# ideally, the names would all be formatted the same and reliable, allowing us to just index into a pandas column. 
# however, they're not, so we have to use this approach, and attempt to replace unreliable ones.

# find the list elements where the agency full name was not available (i.e. worst case), and thus set to NAN (which was then replaced with 0)
grantsDF['AgencyName'].loc[grantsDF['AgencyName'].eq(0)]='Other'
# this will serve as a backstop in the event no information was provided

# in the event no agency code is provided (and thus set to 0) use the capital letters of the agency name
grantsDF['AgencyCode'].loc[grantsDF['AgencyCode'].eq(0)]=grantsDF['AgencyName'].loc[grantsDF['AgencyCode'].eq(0)].map(lambda x: ''.join([char for char in x if char.isupper()]))

# however, we don't want to have surreptitiously created the 'O' agency, so set 'O' to other, instead of the abbreviation
grantsDF['AgencyCode'].loc[grantsDF['AgencyCode'].eq('O')]='Other'

# also remove everything after the first hyphen.  We dont need to go that far down for this in the hirearchy
grantsDF['AgencyCode']=grantsDF['AgencyCode'].map(lambda x: x.split('-',1)[0])

# let's save down this fixed dataframe to speed up later analyses, 
# NOTE can't use comma as separator due to Description content.  Use Bar instead?
grantsDF.to_csv('allGrantsData.csv', sep='|')

,OpportunityID,AgencyName,EstimatedTotalProgramFunding,AwardCeiling,AwardFloor,ExpectedNumberOfAwards
12425,287495,Department of Housing and Urban Development,1900000000,1500000,2500,8000
66712,251274,Department of Homeland Security - FEMA,32092008,0,0,10000
17758,295436,National Park Service,26965,0,0,26965
6580,279192,National Park Service,1,0,0,27240
19056,297091,Fish and Wildlife Service,0,30000,0,30000
4533,276884,Fish and Wildlife Service,0,1,0,30740
6115,278651,National Park Service,1,0,0,36698
27033,175873,Bureau of Land Management,40000,40000,10000,40000
43664,60036,National Park Service,1,0,0,44471
10751,284299,National Park Service,45345,1,1,45345


C:\Users\dbullock\AppData\Local\Temp\1\ipykernel_38712\521758451.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grantsDF['AgencyName'].loc[grantsDF['AgencyName'].eq(0)]='Other'
C:\Users\dbullock\AppData\Local\Temp\1\ipykernel_38712\521758451.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grantsDF['AgencyCode'].loc[grantsDF['AgencyCode'].eq(0)]=grantsDF['AgencyName'].loc[grantsDF['AgencyCode'].eq(0)].map(lambda x: ''.join([char for char in x if char.isupper()]))
C:\Users\dbullock\AppData\Local\Temp\1\ipykernel_38712\521758451.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

### Quick inspection of the dataset

Now that the data has been downloaded and cleaned, let's take a moment to take a look at the the broad scope of the database.  To do this, we'll look at it from an agency-based perspective, and see how many grants are recorded as well as what their total funding is.  

Keep in mind that, although this is a fairly comprehensive database, it may not include all grants, and even the grant records it _does_ contain may all not be formatted in a standard way (and thus may be overlooked by the method we employ here).

We'll also save down this output to the working directory, in a file named `agencyGrantsSummary.csv`.  

In [10]:
# we can go ahead and compute the frequencies for these as well
# get the unique entries and their counts
unique_elements, counts_elements = np.unique(list(grantsDF['AgencyCode'].values), return_counts=True)
#get the proportions thereof
countPortions=np.divide(counts_elements,np.sum(counts_elements))

#Now that we have the names, we can obtain other information using this same approach
 
#seems that the data can be unreliable, particualrly when we implement the grant funding estimate that comes up shortly
sanityGrantThresh=2000
    
#do the same but for the grant values
grantValVec=[[] for iGrant in range(grantsDF.shape[0]) ]
for iIndex,iListing in grantsDF.iterrows():
        # place the current estimate info in a variable
        currEstimate=np.int64(iListing['EstimatedTotalProgramFunding']) 
        # if it's not == 0, set it as the value
        if not currEstimate == 0: 
            grantValVec[iIndex]=currEstimate
        # otherwise use the ceiling and floor
        # NOTE: in the event that the ceiling and floor won't provided, they will have been NANs in the initial DataFrame
        # this was then replaced with 0
        # the mean of 0 * 1 = 0, so the following is mathematically fine
        else:
            # take the mean and use that times the number of potential awards
            # if the number of expected awards is 0 or too large, treat it as 1
            if iListing['ExpectedNumberOfAwards'] == 0 or np.int64(iListing['ExpectedNumberOfAwards']) >= sanityGrantThresh :
                expectedAwards=1
                #otherwise use the information provided
            else: 
                expectedAwards=np.int64(iListing['ExpectedNumberOfAwards'])
                
            grantValVec[iIndex]=np.mean([np.int64(iListing['AwardCeiling']),np.int64(iListing['AwardFloor'])])*expectedAwards

# if you ever want to demonstrate why you ought to do sanity checks, try this without setting the sanity grantNum threshold:
#GrantValOrder=np.flip(np.argsort(grantValVec))
#grantsDF[['OpportunityID','AgencyName','EstimatedTotalProgramFunding','AwardCeiling','AwardFloor','ExpectedNumberOfAwards']].iloc[GrantValOrder[0:5
#
# also you can use this to check and see where the 2000 threshold came from
# grantsDF[['AwardCeiling','AwardFloor','EstimatedTotalProgramFunding','ExpectedNumberOfAwards']]=grantsDF[['AwardCeiling','AwardFloor','EstimatedTotalProgramFunding','ExpectedNumberOfAwards']].astype(np.int64)
# (grantsDF[['OpportunityID','AgencyName','EstimatedTotalProgramFunding','AwardCeiling','AwardFloor','ExpectedNumberOfAwards']].sort_values(by='ExpectedNumberOfAwards')).iloc[-100:-50]
# Note that those grants that DO have accurate grant numbers in this range, have an accurate estimated award.
# this means the loop never gets this far in such a case.

#use that information to get the total value for each agency
#initialize a vector for the totals
agencyTotals=np.zeros(len(unique_elements))
for iAgencyIndex,iUniqueAgencies in enumerate(unique_elements):
    agencyVecMask=[iUniqueAgencies==iAgencies for iAgencies in list(grantsDF['AgencyCode'].values)]
    agencyTotals[iAgencyIndex]=np.sum(list(compress(grantValVec,agencyVecMask)))

#get the proportions thereof
valueProportion=np.divide(agencyTotals,np.sum(agencyTotals))
            
#initialize the dataframe
grantCountDF=pd.DataFrame(data=zip(unique_elements,counts_elements,countPortions,agencyTotals,valueProportion),columns=['AgencyName','GrantCount','PortionOfTotal','TotalValue','TotalValuePortion'])

# save it down
grantCountDF.to_csv('../agencyGrantsSummary.csv')
# interactive display, we'll display 100 MB worth, max
pd.options.display.float_format = '{:,.5f}'.format
grantCountDF                

,AgencyName,GrantCount,PortionOfTotal,TotalValue,TotalValuePortion
0,AC,14,0.00020,"36,340,000.00000",0.00003
1,CNCS,103,0.00146,"446,533,570.00000",0.00034
2,CPSC,9,0.00013,"14,900,005.00000",0.00001
3,DC,8,0.00011,"53,320,000.00000",0.00004
4,DHS,1373,0.01952,"134,645,308,722.50000",0.10191
5,DOC,1168,0.01661,"14,636,620,100.50000",0.01108
6,DOD,3474,0.04940,"56,181,063,624.50000",0.04252
7,DOE,1426,0.02028,"56,439,521,213.00000",0.04272
8,DOI,19804,0.28159,"27,190,785,173.50000",0.02058
9,DOL,609,0.00866,"51,614,437,185.00000",0.03906


### What about open-science-specific grants?

In this notebook we have looked at the data contained within the [grants.gov](https://www.grants.gov/web/grants) export.  In the next notebook ('Open Science Overview'), we'll take a first look at this data more specifically through the lense of open-science infrastructure.